<a href="https://colab.research.google.com/github/1st9aram/dacon_house_prices/blob/main/rf_ng_ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import pandas as pd
import numpy as np

In [ ]:
os.chdir("/content/drive/MyDrive/Colab Notebooks/dacon/house_prices_regression/")
print(os.getcwd())

/content/drive/MyDrive/Colab Notebooks/dacon/house_prices_regression


In [ ]:
train = pd.read_csv('./housing/train.csv')
test = pd.read_csv('./housing/test.csv')
submission = pd.read_csv('./housing/sample_submission.csv')

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1350 non-null   int64 
 1   Overall Qual    1350 non-null   int64 
 2   Gr Liv Area     1350 non-null   int64 
 3   Exter Qual      1350 non-null   object
 4   Garage Cars     1350 non-null   int64 
 5   Garage Area     1350 non-null   int64 
 6   Kitchen Qual    1350 non-null   object
 7   Total Bsmt SF   1350 non-null   int64 
 8   1st Flr SF      1350 non-null   int64 
 9   Bsmt Qual       1350 non-null   object
 10  Full Bath       1350 non-null   int64 
 11  Year Built      1350 non-null   int64 
 12  Year Remod/Add  1350 non-null   int64 
 13  Garage Yr Blt   1350 non-null   int64 
 14  target          1350 non-null   int64 
dtypes: int64(12), object(3)
memory usage: 158.3+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1350 entries, 0 to 1349
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   id              1350 non-null   int64 
 1   Overall Qual    1350 non-null   int64 
 2   Gr Liv Area     1350 non-null   int64 
 3   Exter Qual      1350 non-null   object
 4   Garage Cars     1350 non-null   int64 
 5   Garage Area     1350 non-null   int64 
 6   Kitchen Qual    1350 non-null   object
 7   Total Bsmt SF   1350 non-null   int64 
 8   1st Flr SF      1350 non-null   int64 
 9   Bsmt Qual       1350 non-null   object
 10  Full Bath       1350 non-null   int64 
 11  Year Built      1350 non-null   int64 
 12  Year Remod/Add  1350 non-null   int64 
 13  Garage Yr Blt   1350 non-null   int64 
dtypes: int64(11), object(3)
memory usage: 147.8+ KB


In [ ]:
train = train.iloc[:, 1:]
test = test.iloc[:, 1:]

In [ ]:
train.describe()

,Overall Qual,Gr Liv Area,Garage Cars,Garage Area,Total Bsmt SF,1st Flr SF,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
count,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000
mean,6.208889,1513.542222,1.870370,502.014815,1082.644444,1167.474074,1.560741,1972.987407,1985.099259,1978.471852,186406.312593
std,1.338015,487.523239,0.652483,191.389956,384.067713,375.061407,0.551646,29.307257,20.153244,25.377278,78435.424758
min,2.000000,480.000000,1.000000,100.000000,105.000000,480.000000,0.000000,1880.000000,1950.000000,1900.000000,12789.000000
25%,5.000000,1144.000000,1.000000,368.000000,816.000000,886.250000,1.000000,1955.000000,1968.000000,1961.000000,135000.000000
50%,6.000000,1445.500000,2.000000,484.000000,1009.000000,1092.500000,2.000000,1976.000000,1993.000000,1978.500000,165375.000000
75%,7.000000,1774.500000,2.000000,588.000000,1309.500000,1396.500000,2.000000,2002.000000,2004.000000,2002.000000,217875.000000
max,10.000000,4476.000000,5.000000,1488.000000,2660.000000,2898.000000,4.000000,2010.000000,2010.000000,2207.000000,745000.000000


In [ ]:
test.describe()

,Overall Qual,Gr Liv Area,Garage Cars,Garage Area,Total Bsmt SF,1st Flr SF,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt
count,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000,1350.000000
mean,6.237037,1524.071111,1.871852,499.867407,1101.222222,1175.714815,1.597778,1973.537778,1985.258519,1978.457037
std,1.370057,514.410852,0.651639,186.267579,431.641042,406.858231,0.549002,29.799448,20.818442,25.771750
min,2.000000,407.000000,1.000000,160.000000,173.000000,407.000000,0.000000,1879.000000,1950.000000,1895.000000
25%,5.000000,1169.750000,1.000000,365.250000,825.250000,887.250000,1.000000,1956.000000,1966.000000,1961.000000
50%,6.000000,1472.500000,2.000000,484.000000,1024.000000,1098.000000,2.000000,1976.000000,1995.000000,1982.000000
75%,7.000000,1742.250000,2.000000,578.750000,1338.750000,1409.500000,2.000000,2002.000000,2004.000000,2002.000000
max,10.000000,5642.000000,4.000000,1418.000000,6110.000000,5095.000000,4.000000,2010.000000,2010.000000,2010.000000


In [ ]:
train[train['Garage Yr Blt'] > 2050]

,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
254,8,1564,Gd,2,502,Ex,1546,1564,Gd,2,2006,2007,2207,267300


In [ ]:
train.loc[254, 'Garage Yr Blt'] = 2007

In [ ]:
qual_cols = train.dtypes[train.dtypes == np.object].index

def label_encoder(df_, qual_cols):
    df = df_.copy()
    mapping = {'Ex':5, 'Gd':4, 'TA':3, 'Fa':2, 'Po':1}

    for col in qual_cols:
        df[col] = df[col].map(mapping)
    return df

In [ ]:
train = label_encoder(train, qual_cols)
test = label_encoder(test, qual_cols)

In [ ]:
train

,Overall Qual,Gr Liv Area,Exter Qual,Garage Cars,Garage Area,Kitchen Qual,Total Bsmt SF,1st Flr SF,Bsmt Qual,Full Bath,Year Built,Year Remod/Add,Garage Yr Blt,target
0,10,2392,5,3,968,5,2392,2392,5,2,2003,2003,2003,386250
1,7,1352,4,2,466,4,1352,1352,5,2,2006,2007,2006,194000
2,5,900,3,1,288,3,864,900,3,1,1967,1967,1967,123000
3,5,1174,3,2,576,4,680,680,3,1,1900,2006,2000,135000
4,7,1958,4,3,936,4,1026,1026,4,2,2005,2005,2005,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1345,6,1756,4,2,422,3,872,888,5,2,1996,1997,1996,204000
1346,9,2748,4,3,850,5,1850,1850,5,2,2006,2006,2006,390000
1347,5,1214,3,1,318,3,1214,1214,3,2,1967,1967,1967,143000
1348,5,894,3,2,440,3,864,894,4,1,1974,1974,1974,131000


In [ ]:
train_x = train.drop(['target'], axis = 1)
train_y = np.log1p(train.target)

target = test[train_x.columns]

In [ ]:
target.fillna(target.mean(), inplace = True)

In [ ]:
!pip install ngboost
!pip install catboost

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from ngboost import NGBRegressor
from catboost import Pool
from sklearn.metrics import make_scorer
from sklearn.model_selection import KFold

In [ ]:
def NMAE(true, pred):
    mae = np.mean(np.abs(true-pred))
    score = mae / np.mean(np.abs(true))
    return score

In [ ]:
nmae_score = make_scorer(NMAE, greater_is_better = False)
kf = KFold(n_splits = 10, random_state = 44, shuffle = True)

In [ ]:
rf_pred = np.zeros(target.shape[0])
rf_val = []

for n, (train_idx, val_idx) in enumerate(kf.split(train_x, train_y)) :

    print(f'{n + 1} FOLD Training.....')
    
    tr_x, tr_y = train_x.iloc[train_idx], train_y.iloc[train_idx]
    val_x, val_y = train_x.iloc[val_idx], np.expm1(train_y.iloc[val_idx])
    
    rf = RandomForestRegressor(random_state = 44, criterion = 'mae')
    rf.fit(tr_x, tr_y)
    
    val_pred = np.expm1(rf.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    rf_val.append(val_nmae)

    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    fold_pred = rf.predict(target) / 10
    rf_pred += fold_pred

print(f'10FOLD Mean of NMAE = {np.mean(rf_val)} & std = {np.std(rf_val)}')

1 FOLD Training.....


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,


1 FOLD NMAE = 0.08734158240358587

2 FOLD Training.....


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,


2 FOLD NMAE = 0.09064716310292091

3 FOLD Training.....


...

8 FOLD NMAE = 0.09982868147391834

9 FOLD Training.....


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,


9 FOLD NMAE = 0.08970800408680307

10 FOLD Training.....


/usr/local/lib/python3.7/dist-packages/sklearn/ensemble/_forest.py:407: FutureWarning: Criterion 'mae' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='absolute_error'` which is equivalent.
  FutureWarning,


10 FOLD NMAE = 0.1029577880026904

10FOLD Mean of NMAE = 0.09911193332810761 & std = 0.008994899240859234


In [ ]:
ngb_pred = np.zeros(target.shape[0])
ngb_val = []

for n, (tr_idx, val_idx) in enumerate(kf.split(train_x, train_y)) :

    print(f'{n + 1} FOLD Training.....')

    tr_x, tr_y = train_x.iloc[train_idx], train_y.iloc[train_idx]
    val_x, val_y = train_x.iloc[val_idx], np.expm1(train_y.iloc[val_idx])
    
    ngb = NGBRegressor(random_state = 44, n_estimators = 1000, verbose = 0, learning_rate = 0.02)
    ngb.fit(tr_x, tr_y, val_x, val_y, early_stopping_rounds = 300)
    
    val_pred = np.expm1(ngb.predict(val_x))
    val_nmae = NMAE(val_y, val_pred)
    ngb_val.append(val_nmae)

    print(f'{n + 1} FOLD NMAE = {val_nmae}\n')
    
    target_data = Pool(data = target, label = None)
    fold_pred = ngb.predict(target) / 10
    ngb_pred += fold_pred

print(f'10FOLD Mean of NMAE = {np.mean(ngb_val)} & std = {np.std(ngb_val)}')

1 FOLD Training.....
1 FOLD NMAE = 0.06430021555642125

2 FOLD Training.....
2 FOLD NMAE = 0.07190297997543939

3 FOLD Training.....
3 FOLD NMAE = 0.08267222656271835

4 FOLD Training.....
4 FOLD NMAE = 0.07844862424930737

5 FOLD Training.....
5 FOLD NMAE = 0.06381058765510395

6 FOLD Training.....
6 FOLD NMAE = 0.08145264558616122

7 FOLD Training.....
7 FOLD NMAE = 0.07357168094833062

8 FOLD Training.....
8 FOLD NMAE = 0.07257910939957507

9 FOLD Training.....
9 FOLD NMAE = 0.07219725527421232

10 FOLD Training.....
10 FOLD NMAE = 0.10087869910030217

10FOLD Mean of NMAE = 0.07618140243075715 & std = 0.010164539872514635


In [ ]:
(rf_pred + ngb_pred) / 2

array([12.71833951, 11.78053832, 12.07345545, ..., 11.30386003,
       12.20639813, 11.83350227])

In [ ]:
submission['target'] = np.expm1((rf_pred + ngb_pred) / 2)

In [ ]:
submission.to_csv('./submission_4.csv', index=False)